Put some basic use cases for automated backtest/trading

In [ ]:
import twsq
from twsq.alpha import Alpha
import pandas as pd
import numpy as np

In [ ]:
class ETHDCA(Alpha):

    def rebalance(self):
        self.create_order(
            'ETH/USD',
            1,
            'buy',
            route=True)
            
        return  

In [6]:
result = ETHDCA.run_backtest(start_ts='20210101')

ExchangeNotAvailable: binance GET https://api.binance.com/api/v3/exchangeInfo 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}